In [128]:
import pandas as pd
import datetime
coviddata=pd.read_hdf('./covidtata.h5',key='data1129')

#get daily covid cases
coviddata['new_cases_daily']=coviddata['新增无症状']+coviddata['新增本土确诊']
df_covid=coviddata[['省份', '城市', '日期','new_cases_daily']]

s1=df_covid['日期']>"2022-02-01" #2022年2月1号开始区分 新增本土无症状（即表里的新增无症状）和新增本土确诊
s2=df_covid['省份'].isin(['北京','上海','天津','重庆'])
s3=~df_covid['城市'].isin(['北京','上海','天津','重庆'])

df_covid['城市']=df_covid['城市'].str.replace('伊犁州','伊犁')

df_covid=df_covid[s1] #去掉2022年前的
df_covid=df_covid[~(s2&s3)] #去掉直辖市分区的统计，因为后面没抓到其实，而且分析也没有意义

df_covid['日期'] = pd.to_datetime(df_covid['日期']).dt.date

def getyr(d):
    return str(d.isocalendar()[0])

df_covid['yr']=df_covid['日期'].apply(getyr)


#rolling average
df_covid=df_covid.sort_values(by='日期',ascending=True).reset_index(drop=True)
df_covid['cases_last7d_max'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 1).max())

df_covid=df_covid.rename(columns={'日期':'date'})

df_covid



C:\Users\Leo\AppData\Local\Temp\ipykernel_3360\2006568066.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['城市']=df_covid['城市'].str.replace('伊犁州','伊犁')
C:\Users\Leo\AppData\Local\Temp\ipykernel_3360\2006568066.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_covid=df_covid[~(s2&s3)] #去掉直辖市分区的统计，因为后面没抓到其实，而且分析也没有意义


,省份,城市,date,new_cases_daily,yr,cases_last7d_max
0,宁夏,中卫,2022-02-02,0,2022,0.0
1,贵州,毕节,2022-02-02,0,2022,0.0
2,河南,驻马店,2022-02-02,0,2022,0.0
3,广西,防城港,2022-02-02,0,2022,0.0
4,河南,南阳,2022-02-02,0,2022,0.0
...,...,...,...,...,...,...
104031,河南,周口,2022-11-28,0,2022,0.0
104032,广西,钦州,2022-11-28,2,2022,7.0
104033,广西,南宁,2022-11-28,23,2022,47.0
104034,甘肃,金昌,2022-11-28,0,2022,6.0


In [21]:
pd.read_excel('./Amap Conjestion.xlsx').to_hdf('./amap.h5',key='raw1129')

In [129]:
# get 7d rolling amap traffic
amap=pd.read_hdf('./amap.h5',key='raw1129')
amap=pd.melt(amap,id_vars=['城市','省','城市线'],var_name='date', value_name='traffic')
amap['date']=pd.to_datetime(amap['date']).dt.date


def getyr(d):
    return str(d.isocalendar()[0])
def getdt(d):
    return str(d)[-5:]

amap['yr']=amap['date'].apply(getyr)
amap['dt']=amap['date'].apply(getdt)

#rolling average
amap=amap.sort_values(by='date',ascending=True).reset_index(drop=True)

amap_2019=amap[amap['yr'].str.contains('2019')]
amap_2019['traffic_last7d_mean'] = amap_2019.groupby('城市')['traffic'].transform(lambda x: x.rolling(30, 1).mean())
amap_2019=amap_2019[['省','城市','dt','traffic_last7d_mean']]
amap_2019=amap_2019.rename(columns={"traffic_last7d_mean":"traffic_19_rolling_mean"})

amap_2022=amap[amap['yr'].str.contains('2022')]
amap_2022=amap_2022.merge(amap_2019,how='left',on=['省','城市','dt'])
amap_2022['traffic_last7d_mean'] = amap_2022.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 1).mean())

amap_2022

C:\Users\Leo\AppData\Local\Temp\ipykernel_3360\2672002282.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amap_2019['traffic_last7d_mean'] = amap_2019.groupby('城市')['traffic'].transform(lambda x: x.rolling(30, 1).mean())


,城市,省,城市线,date,traffic,yr,dt,traffic_19_rolling_mean,traffic_last7d_mean
0,德阳,四川,三线及以下,2022-01-03,1.52,2022,01-03,1.686667,1.520000
1,江门,广东,三线及以下,2022-01-03,1.41,2022,01-03,1.466667,1.410000
2,济宁,山东,三线及以下,2022-01-03,1.37,2022,01-03,1.743333,1.370000
3,淮安,江苏,三线及以下,2022-01-03,1.30,2022,01-03,1.460000,1.300000
4,湖州,浙江,三线及以下,2022-01-03,1.30,2022,01-03,1.336667,1.300000
...,...,...,...,...,...,...,...,...,...
32995,青岛,山东,二线,2022-11-28,1.86,2022,11-28,1.734000,1.697143
32996,苏州,江苏,二线,2022-11-28,1.75,2022,11-28,1.508333,1.524286
32997,杭州,浙江,二线,2022-11-28,1.57,2022,11-28,1.533667,1.470000
32998,潍坊,山东,二线,2022-11-28,1.26,2022,11-28,1.586333,1.360000


In [132]:
import numpy as np

city_popu=pd.read_hdf('./citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

dfo=df_covid.merge(city_popu,on='城市',how='inner')

# c1=~dfo['城市'].isin(['伊犁','中国香港'])
# dfo=dfo[c1]

def catgorize(num):
    num=float(num)
    level=-2
    if num ==0:
        level=-1
    if 0 < num <=1:
        level =0
    if num > 1 and num <=10:
        level =0
    if num > 10 and num <=100:
        level =1
    if num > 100 and num <=500:
        level =2
    if num > 500 and num <=1000:
        level =2
    if num > 1000:
        level =2
    return level


dfo['covid_level']=dfo['cases_last7d_max'].apply(catgorize)


show=dfo.copy()
c1=show['covid_level']>-2
show=show[c1]





dfo=amap_2022.merge(show,on=['城市','yr','date'],how='inner')
dfo['traffic_change']=dfo['traffic_last7d_mean']/dfo['traffic_19_rolling_mean']-1

# dfo





# pd.pivot_table(show, values='常住人口', index=['date'],columns=['covid_level'], aggfunc=np.sum).to_clipboard()

# dfo
show2=pd.pivot_table(dfo, values='traffic_change', index=['date'],columns=['covid_level'], aggfunc=np.mean).reset_index()
show2[1][show2[1]>0]=np.nan
show2=show2[show2['date']>=datetime.date(2022,3,1)]
show2.to_clipboard(index=False)


# # topivot=show[c1&c2].groupby(by=['wk',show_value]).count().reset_index()
# # topivot2=topivot.merge(dates,how='left',on='wk')
# # pd.pivot_table(topivot2, values='traffic_change', index=['日期'],columns=[show_value], aggfunc=np.sum).to_clipboard()



C:\Users\Leo\AppData\Local\Temp\ipykernel_3360\4210941797.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show2[1][show2[1]>0]=np.nan


In [95]:
l1=df_covid['城市'].drop_duplicates()
l2=city_popu['城市'].drop_duplicates()
l2.to_clipboard()

In [131]:
l1=dfo['城市'].drop_duplicates()
l2=amap['城市'].drop_duplicates()

[i for i in list(l2) if i not in list(l1)]

['中国香港']

In [134]:
dfo.to_clipboard()